In [10]:
from load_preprocess.functions import Data
import pandas as pd
import warnings
warnings.filterwarnings("ignore")
# import config
import json

In [ ]:
path_conf = "params/config.json"
conf = json.load(open(path_conf, "r"))
locals().update(conf)

In [2]:
data = Data(path = 'data/')
df = data.create_df()

100%|██████████| 8/8 [00:47<00:00,  5.97s/it]


In [ ]:
df.head()

,region,road_density,length_m,area_m,length_max,length_mean
3,Île-de-France,0.000227,2.744410e+06,1.206353e+10,85932.399845,3076.692877
15,Nord-Pas-de-Calais,0.000151,1.887368e+06,1.249483e+10,147881.271737,4015.676440
10,Haute-Normandie,0.000113,1.398489e+06,1.237197e+10,135138.054662,4994.602982
0,Alsace,0.000106,8.836421e+05,8.322648e+09,171172.687144,4374.466068
17,Picardie,0.000093,1.822331e+06,1.954720e+10,163238.986421,7854.875311


In [ ]:
df_on = pd.read_excel(r'data/road_go_na_ru3g_spreadsheet.xlsx', sheet_name='Sheet 1', skiprows=8)
df_off = pd.read_excel(r'data/road_go_na_rl3g_spreadsheet.xlsx', sheet_name='Sheet 1', skiprows=8)

def clean_freight_df(df, on_load=True):
    
    df = df[['TIME', 'TIME.1', '2021']]
    
    if on_load:
        freight_type = "number_onload"
    else:
        freight_type = "number_offload"
        
    df.rename(columns={"TIME": "geo_code", "TIME.1": "geo_labels", "2021": freight_type}, inplace=True)
    df = df.iloc[1:]
    df_clean = df[df.geo_code.str.startswith("FR").fillna(False)]
    df_clean.reset_index(inplace=True, drop=True)
    
    df_clean[freight_type] = df_clean[freight_type].astype('str')
    df_clean[freight_type] = df_clean[freight_type].str.replace('\.0*$', '', regex=True)
    df_clean.loc[~(df_clean[freight_type].str.isdigit()), freight_type] = '0'
    df_clean[freight_type] = df_clean[freight_type].astype('int')
    
    return df_clean

def department_region_map(path, df_fr):
    df_dpts_region = pd.read_csv(path)
    df_dpts_region.rename(columns={"dep_name":"geo_labels", "region_name":"new_region_name", "old_region_name": "region"}, inplace=True)
    df_final = pd.merge(df_fr, df_dpts_region[["geo_labels", "region"]], how='left', on=['geo_labels'])
    df_final = df_final[~(df_final.geo_labels.str.endswith(" "))]
    return df_final

def merge_freight(path, df_on, df_off):
    
    df_onload_fr = clean_freight_df(df_on, on_load=True)
    df_offload_fr = clean_freight_df(df_off, on_load=False)

    df_fr = pd.merge(df_onload_fr, df_offload_fr, how='inner', on=['geo_code', 'geo_labels'])
    df_fr['total_load'] = df_fr.number_offload + df_fr.number_onload 
    
    df_fr['geo_labels'] = [c[0] for c in df_fr['geo_labels'].str.split("(")]
    df_fr = department_region_map(path, df_fr)
    df_fr = df_fr.groupby("region")["total_load"].sum().reset_index()
    df_fr["full_load"] = df_fr["total_load"].sum()   
    df_fr["perc_load"] = df_fr["total_load"]/df_fr["full_load"]

    return df_fr

path_region_dpt_map = 'data/departements-region.csv'
df_fr = merge_freight(path_region_dpt_map, df_on, df_off)

df_fr.head()

df_new = pd.merge(df, df_fr[["region", "perc_load"]], how="left", on="region")
df_fr.head()

,region,total_load,full_load,perc_load
0,Alsace,96286,3213074,0.029967
1,Aquitaine,188556,3213074,0.058684
2,Auvergne,76627,3213074,0.023849
3,Basse-Normandie,78563,3213074,0.024451
4,Bourgogne,95622,3213074,0.029760
5,Bretagne,213487,3213074,0.066443
6,Centre,149795,3213074,0.046620
7,Champagne-Ardenne,112517,3213074,0.035018
8,Corse,8148,3213074,0.002536
9,Franche-Comté,65182,3213074,0.020286


In [ ]:
# In Config file now:

# H2_trucks_2030 = 10000
# H2_trucks_2040 = 60000
# # https://www.eea.europa.eu/data-and-maps/daviz/hdv-market-share-for-vehicle#tab-chart_1
# autonomy_high_ms = 0.46
# autonomy_medium_ms = 0.26
# autonomy_low_ms = 0.28
# autonomy_high_km = 1000
# autonomy_medium_km = 400
# autonomy_low_km = 150
# max_hours_drive = 9
# avg_speed_kmh = 80
# avg_time_fill = 15/60
# open_time = 24

In [ ]:
def calculate_number_stations(df):
    
    df["max_length_drive"] = max_hours_drive*avg_speed_kmh
    df["length_max"] = df["length_max"]/1000
    df["length_mean"] = df["length_mean"]/1000
    df["avg_distance_high_aut"] = df[["max_length_drive", "length_max"]].min(axis=1)
    df["avg_distance_midlow_aut"] = 0.6*df["avg_distance_high_aut"]#df_new[["max_length_drive", "length_mean"]].min(axis=1)
    df["h2_num_2030"] = H2_trucks_2030*df["perc_load"]
    df["R_2030_high_aut"] = autonomy_high_ms*df["h2_num_2030"]*df["avg_distance_high_aut"]/autonomy_high_km
    df["R_2030_mid_aut"] = autonomy_medium_ms*df["h2_num_2030"]*df["avg_distance_midlow_aut"]/autonomy_medium_km
    df["R_2030_low_aut"] = autonomy_low_ms*df["h2_num_2030"]*df["avg_distance_midlow_aut"]/autonomy_low_km
    df["R_2030_total"] = df["R_2030_high_aut"] + df["R_2030_mid_aut"] + df["R_2030_low_aut"]
    df["C_2030"] = open_time/avg_time_fill
    df["num_stations"] = df["R_2030_total"] / df["C_2030"]

    return df

In [ ]:
df_new = calculate_number_stations(df_new)

,region,road_density,length_m,area_m,length_max,length_mean,perc_load,max_length_drive,avg_distance_high_aut,avg_distance_midlow_aut,h2_num_2030,R_2030_high_aut,R_2030_mid_aut,R_2030_low_aut,R_2030_total,C_2030,num_stations
0,Île-de-France,0.000227,2.744410e+06,1.206353e+10,85.932400,3.076693,0.097211,720,85.932400,51.559440,972.112687,38.426549,32.579031,93.560293,164.565873,96.0,1.714228
1,Nord-Pas-de-Calais,0.000151,1.887368e+06,1.249483e+10,147.881272,4.015676,0.066784,720,147.881272,88.728763,667.843318,45.430299,38.516992,110.612901,194.560193,96.0,2.026669
2,Haute-Normandie,0.000113,1.398489e+06,1.237197e+10,135.138055,4.994603,0.039934,720,135.138055,81.082833,399.337208,24.824201,21.046605,60.441532,106.312337,96.0,1.107420
3,Alsace,0.000106,8.836421e+05,8.322648e+09,171.172687,4.374466,0.029967,720,171.172687,102.703612,299.669413,23.595801,20.005135,57.450645,101.051581,96.0,1.052621
4,Picardie,0.000093,1.822331e+06,1.954720e+10,163.238986,7.854875,0.040049,720,163.238986,97.943392,400.491865,30.072908,25.496596,73.220993,128.790496,96.0,1.341568
5,Rhône-Alpes,0.000090,4.039422e+06,4.497380e+10,203.474560,4.047518,0.113242,720,203.474560,122.084736,1132.423343,105.992897,89.863543,258.069663,453.926103,96.0,4.728397
6,Lorraine,0.000085,2.007468e+06,2.366142e+10,189.457934,4.562428,0.030442,720,189.457934,113.674761,304.415647,26.530021,22.492844,64.594835,113.617700,96.0,1.183518
7,Bretagne,0.000085,2.333729e+06,2.760417e+10,244.626451,4.037593,0.066443,720,244.626451,146.775871,664.432254,74.767344,63.389705,182.042229,320.199278,96.0,3.335409
8,Basse-Normandie,0.000078,1.386823e+06,1.780635e+10,122.829921,3.688359,0.024451,720,122.829921,73.697952,244.510397,13.815269,11.712945,33.637176,59.165390,96.0,0.616306
9,Pays de la Loire,0.000077,2.480511e+06,3.233398e+10,228.887325,6.852239,0.078611,720,228.887325,137.332395,786.113236,82.768424,70.173229,201.523118,354.464770,96.0,3.692341
